In [ ]:
# Requirements: python >= 3.0; Pytorch >= 2.0.0 

# pip install TotalSegmentator 

# Next, please get an academic license for TotalSegmentator (we need it to process the task "appendicular_bones") - should be done in a few seconds! 
# https://backend.totalsegmentator.com/license-academic/
# Original GitHub site containing license information: 
# https://github.com/wasserth/TotalSegmentator#:~:text=Available%20with%20a%20license%20(free%20licenses%20available%20for%20non%2Dcommercial%20usage%20here.%20For%20a%20commercial%20license%20contact%20jakob.wasserthal%40usb.ch)%3A

# Set your license, following email instructions 
# totalseg_set_license -l <your-license-number>

# Download weights of the pre-trained model
# totalseg_download_weights -t total [femur_left, femur_right]
# totalseg_download_weights -t appendicular_bones [patella, tibia, fibula]

In [ ]:
import time
import subprocess
import numpy as np
import SimpleITK as sitk 

from scipy.ndimage import uniform_filter
from skimage.filters import threshold_otsu

# Input and output directories
ct_dir = "/mnt/c/users/avery/Desktop/PI201/DICOM/P0000001/ST000001/SE000003"  # Input CT directory 
seg_dir = "/mnt/c/users/avery/Desktop/segmentation_masks"  # Output segmentation masks directory

In [17]:
# CT
reader = sitk.ImageSeriesReader()
series_ids = reader.GetGDCMSeriesIDs(ct_dir)
if not series_ids:
    raise ValueError(f"No DICOM series found in directory: {ct_dir}")
series_file_names = reader.GetGDCMSeriesFileNames(ct_dir, series_ids[0])
reader.SetFileNames(series_file_names)
ct = reader.Execute()
print("CT loaded successfully.")

CT loaded successfully.


In [ ]:
# Extract metadata
ct_dims = ct.GetSize()         # Returns a tuple, e.g., (width, height, depth)
ct_spacing = ct.GetSpacing()     # Returns voxel spacing, e.g., (spacing_x, spacing_y, spacing_z)
ct_origin = ct.GetOrigin()       # Returns the origin of the image

# Calculate voxel volume (mm^3)
voxel_volume = ct_spacing[0] * ct_spacing[1] * ct_spacing[2]

print("Image Dimensions (Width, Height, Depth):", ct_dims)
print("Spacing (mm):", ct_spacing)
print("Origin:", ct_origin)
print("Voxel Volume (mm^3):", voxel_volume)

Image Dimensions (Width, Height, Depth): (512, 512, 251)
Spacing (mm): (0.485, 0.485, 1.0)
Origin: (-147.882, -125.382, 892.5)
Voxel Volume (mm^3): 0.235225


In [4]:
# TotalSegmentator 

# Commands to run segmentations for trained classes 
command_total = f"TotalSegmentator -i {ct_dir} -o {seg_dir} --ta total" # "total"
command_appendicular = f"TotalSegmentator -i {ct_dir} -o {seg_dir} --ta appendicular_bones" # "appendicular_bones"

# Record starting time
start_time = time.time()

# Run first command (femurs)
print("Running segmentation with '--ta total'...")
result_total = subprocess.run(command_total, shell=True, capture_output=True, text=True)
if result_total.stderr:
    print("Errors:")
    print(result_total.stderr)
print("Finished '--ta total' segmentation./n")

# Run second command (patella, tibia, fibula)
print("Running segmentation with '--ta appendicular_bones'...")
result_appendicular = subprocess.run(command_appendicular, shell=True, capture_output=True, text=True)
if result_appendicular.stderr:
    print("Errors:")
    print(result_appendicular.stderr)
print("Finished '--ta appendicular_bones' segmentation./n")

# Display time elapsed
elapsed_time = time.time() - start_time
print(f"Total elapsed time: {elapsed_time:.2f} seconds")



Running segmentation with '--ta total'...
Errors:

  0%|          | 0/8 [00:00<?, ?it/s]/home/avery/torchenv/lib/python3.12/site-packages/torch/nn/modules/conv.py:605: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv3d(

100%|██████████| 8/8 [00:03<00:00,  2.03it/s]

100%|██████████| 8/8 [00:00<00:00,  9.32it/s]

100%|██████████| 8/8 [00:00<00:00,  9.19it/s]

100%|██████████| 8/8 [00:00<00:00,  9.56it/s]

100%|██████████| 8/8 [00:00<00:00, 10.42it/s]

Finished '--ta total' segmentation./n
Running segmentation with '--ta appendicular_bones'...
Errors:

  0%|          | 0/8 [00:00<?, ?it/s]/home/avery/torchenv/lib/python3.12/site-packages/torch/nn/modules/conv.py:605: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv3d(

100%|██████████| 8/8 [00:02<00:00,  2.74it/s]

Finished '--ta 

In [106]:
def refine_mask_adaptive_otsu(mask_path, ct, output_path,
                               window_size=2, dilation_radius=3, erosion_radius=1, min_component_size=500, gaussian_sigma=2.0):
    """
    Refine segmentation using adaptive thresholding with Otsu-derived HU threshold.
    
    Parameters:
        mask_path (str): Path to segmentation mask
        ct_path (str): Path to CT scan in HU
        output_path (str): Where to save the refined mask
        window_size (int): Size of cube for adaptive mean filter
        erosion_radius (int): Core erosion to preserve center
        min_component_size (int): Minimum voxels to keep in connected components
        gaussian_sigma (float): Sigma for smoothing CT before Otsu
    """
    start = time.time()

    # --- Load mask and CT ---
    mask = sitk.ReadImage(mask_path)

    # Resample CT to mask geometry
    resample = sitk.ResampleImageFilter()
    resample.SetReferenceImage(mask)
    resample.SetInterpolator(sitk.sitkNearestNeighbor)
    ct_aligned = resample.Execute(ct)

    # Smooth CT to suppress noise before Otsu
    ct_smoothed = sitk.SmoothingRecursiveGaussian(ct_aligned, gaussian_sigma)
    mask_float = sitk.Cast(mask, sitk.sitkFloat32)
    mask_smoothed = sitk.BinaryThreshold(sitk.SmoothingRecursiveGaussian(mask_float, gaussian_sigma),
                                     lowerThreshold=0.5, upperThreshold=1e9,
                                     insideValue=1, outsideValue=0)
    

    # Convert arrays
    ct_np = sitk.GetArrayFromImage(ct_smoothed)
    mask_np = sitk.GetArrayFromImage(mask)

    # --- Step 1: Define shell for Otsu and adaptive filtering ---
    dilated = sitk.BinaryDilate(mask, [dilation_radius]*3)
    dilated_np = sitk.GetArrayFromImage(dilated)

    # Apply Otsu threshold only inside the dilated mask
    ct_in_dilated = ct_np[dilated_np > 0]
    if ct_in_dilated.size < 20:
        print("Insufficient bone region for Otsu. Skipping.")
        return

    otsu_hu_thresh = threshold_otsu(ct_in_dilated)
    print(f"[Otsu] HU threshold inside boundary: {otsu_hu_thresh:.1f}")

    # --- Step 2: Adaptive local HU filtering ---
    local_mean = uniform_filter(ct_np, size=window_size)
    refined_np = (dilated_np > 0) & (local_mean > otsu_hu_thresh)

    # --- Step 3: Remove speckles ---
    refined_img = sitk.GetImageFromArray(refined_np.astype(np.uint8))
    refined_img.CopyInformation(mask)
    cc = sitk.ConnectedComponent(refined_img)

    label_stats = sitk.LabelShapeStatisticsImageFilter()
    label_stats.Execute(cc)

    cc_np = sitk.GetArrayFromImage(cc)
    cleaned_np = np.zeros_like(refined_np)
    for l in label_stats.GetLabels():
        if label_stats.GetNumberOfPixels(l) > min_component_size:
            cleaned_np[cc_np == l] = 1

    # --- Step 4: Preserve interior core ---
    eroded = sitk.BinaryErode(mask_smoothed, [erosion_radius]*3)
    eroded_np = sitk.GetArrayFromImage(eroded)

    final_combined = np.logical_or(eroded_np, cleaned_np)

    # --- Step 5: Save output ---
    final_img = sitk.GetImageFromArray(final_combined.astype(np.uint8))
    final_img.CopyInformation(mask)
    sitk.WriteImage(final_img, output_path)

    elapsed = time.time() - start
    print(f"[✓] Hybrid Otsu-adaptive refined mask saved to {output_path} | Time: {elapsed:.2f}s")


In [107]:
for m in ["femur_left", "femur_right", "patella", "tibia", "fibula"]:
  refine_mask_adaptive_otsu(f"{seg_dir}/{m}.nii.gz", ct, output_path=f"{seg_dir}/{m}_otsu.nii.gz", window_size=1, dilation_radius=1, erosion_radius=1, min_component_size=100, gaussian_sigma=2)

Insufficient bone region for Otsu. Skipping.
[Otsu] HU threshold inside boundary: 367.5
[✓] Hybrid Otsu-adaptive refined mask saved to /mnt/c/users/avery/Desktop/segmentation_masks/femur_right_otsu.nii.gz | Time: 11.92s
[Otsu] HU threshold inside boundary: 340.2
[✓] Hybrid Otsu-adaptive refined mask saved to /mnt/c/users/avery/Desktop/segmentation_masks/patella_otsu.nii.gz | Time: 12.46s
[Otsu] HU threshold inside boundary: 368.4
[✓] Hybrid Otsu-adaptive refined mask saved to /mnt/c/users/avery/Desktop/segmentation_masks/tibia_otsu.nii.gz | Time: 14.31s
[Otsu] HU threshold inside boundary: 401.4
[✓] Hybrid Otsu-adaptive refined mask saved to /mnt/c/users/avery/Desktop/segmentation_masks/fibula_otsu.nii.gz | Time: 12.56s
